In [3]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=67620b7500c5f476963b4d2d8e0740ed17e36690c69e4c61232a06e33ca6e6d0
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=6f042239aea3c8b33468bfd28b1a337243b511379ae6dba954a70ae5c709124e
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

In [4]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import matplotlib.pyplot as plt

# CUDA kernel code
kernel = """
__global__ void mandelbrot(float *image, int width, int height, float xmin, float xmax, float ymin, float ymax)
{
    int gid_x = blockIdx.x * blockDim.x + threadIdx.x;
    int gid_y = blockIdx.y * blockDim.y + threadIdx.y;
    int offset = gid_x + gid_y * width;
    float x = xmin + (xmax - xmin) * gid_x / width;
    float y = ymin + (ymax - ymin) * gid_y / height;
    float real = x;
    float imag = y;
    float r2, i2, temp;
    int value = 0;
    for (int i = 0; i < 256; i++) {
        r2 = real * real;
        i2 = imag * imag;
        temp = 2 * real * imag + y;
        imag = i2 - r2 + x;
        real = temp;
        if (r2 + i2 > 4.0f) {
            value = i;
            break;
        }
    }
    image[offset] = (float)value / 256.0f;
}
"""

# Compile the kernel
mod = SourceModule(kernel)
mandelbrot = mod.get_function("mandelbrot")

# Set grid and block dimensions
block = (16, 16, 1)
grid = (32, 32)

# Create input and output arrays
width = 1024
height = 1024
image = np.zeros((width * height,), dtype=np.float32)

# Allocate memory on the GPU
image_gpu = cuda.mem_alloc(image.nbytes)

# Copy input data to the GPU
cuda.memcpy_htod(image_gpu, image)

# Execute the kernel
mandelbrot(image_gpu, np.int32(width), np.int32(height), np.float32(-2.0), np.float32(1.0), np.float32(-1.5), np.float32(1.5), block=block, grid=grid)

# Copy the result back to the host
cuda.memcpy_dtoh(image, image_gpu)

# Save the output to file
plt.savefig('fractal.jpg', format='jpg')

<Figure size 432x288 with 0 Axes>